In [ ]:
import pandas as pd
import collections

In [ ]:
df_train=pd.read_excel("train.xlsx")

In [ ]:
m=dict(collections.Counter(df_train["screen_name"]))
sorted(m.items(),key=lambda w:w[1], reverse=True)